In [1]:
#import Module
import ee
import geemap

In [2]:
#Login
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1QEqnvdnWdBJHvOJAZUVjmoOL4nEdLwl7XOsOtaKFoo6pHsuVylY9cY

Successfully saved authorization token.


In [3]:
#Input Variabels
start = input("Tanggal Awal:")
finish = input("Tanggal Akhir:")
bounds = geemap.shp_to_ee(input("Lokasi shp Area:"))


Tanggal Awal:2019-03-30
Tanggal Akhir:2019-12-31
Lokasi shp Area:D:\Cloud Computing\GEE\SHP\TN\TNBTS (dari Kang Hardjono)\RESORT_TNBTS.shp


In [9]:
clip = bounds

In [20]:
#Image Colection1
imagecol = ee.ImageCollection("COPERNICUS/S2").filterDate(start, finish).filterBounds(bounds).filter('CLOUDY_PIXEL_PERCENTAGE < 5').median()

In [5]:
imagecol.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2',
 'version': 1593599720230240.0,
 'properties': {'date_range': [1435017600000, 1593388800000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/sentinel2_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 data contain 13 UINT16 spectral bands representing\nTOA reflectance scaled by 10000. See the <a href="https://sentinel.esa.int/documents/247904/685211/Sentinel-2_User_Handbook">Sentinel-2 User Handbook</a>\nfor details. In addition, three QA bands are present where one\n(QA60) is a bitmask band with cloud mask information. For more\ndetails,

In [14]:
#Raster Calculator
#Indeks Vegetasi Function
def indeksVegetasi(img):
    rumus = input('Masukkan Formula:')
    indeks = img.expression(rumus,
                           {'B1': img.select('B1'),
                            'B2': img.select('B2'),
                            'B3': img.select('B3'), 'B4': img.select('B4'), 'B5': img.select('B5'), 'B6': img.select('B6'),
                            'B7': img.select('B7'), 'B8': img.select('B8'), 'B9': img.select('B9'), 'B10': img.select('B10'),
                            'B11': img.select('B11'), 'B12': img.select('B12'), 'B8A': img.select('B8A'), 
                            'QA10': img.select('QA10'), 'QA20': img.select('QA20'), 'QA60': img.select('QA60')})
    return indeks


In [17]:
NDVI = indeksVegetasi(imagecol)

Masukkan Formula:(B4-B3)/(B4+B3)


In [21]:
#Tile Layers
leftlayer = geemap.ee_tile_layer(imagecol.clip(bounds),
                                 {'bands':['B4', 'B3','B2'],
                                  'min':0, 'max':2500},
                                 name='ImageCollection')
rightlayer = geemap.ee_tile_layer(NDVI.clip(bounds),
                                  {'palette': ['red', 'yellow', 'green']},
                                  name='NDVI')

In [22]:
#Visualisasi Layer Overlay
Map = geemap.Map()
Map.centerObject(bounds, zoom=12)
Map.addLayer(imagecol.clip(bounds),
             {'bands':['B4', 'B3','B2'], 'min':0,'max':2500}, name='ImageCollection')
Map.addLayer(NDVI.clip(bounds),
            {'palette': ['red', 'yellow', 'green']},
            name='NDVI')
Map

Map(center=[-8.033989020419423, 112.93390157219604], controls=(WidgetControl(options=['position'], widget=HBox…

In [35]:
#Visualisasi Split Image dan Indeks Vegetasi
Map = geemap.Map()
Map.centerObject(bounds, zoom=12)
Map.split_map(leftlayer, rightlayer)
Map

Map(center=[-8.033989020419423, 112.93390157219604], controls=(WidgetControl(options=['position'], widget=HBox…

In [41]:
#Variable TimeSeries
sentinel_ts = geemap.sentinel2_timeseries(roi=bounds, start_year=2015, end_year=2020, start_date='04-01', end_date='08-30')

In [42]:
#Penamaan Layer TimeSeries
layer_names = ['Sentinel2_' + str(year) for year in range(2015,2021)]
print(layer_names)

#Parameter Visualisasi
sentinel_vis = {'bands':['Red', 'Green','Blue'], 'min':0,'max':2500}

['Sentinel2_2015', 'Sentinel2_2016', 'Sentinel2_2017', 'Sentinel2_2018', 'Sentinel2_2019', 'Sentinel2_2020']


In [47]:
#Visualiasi Timeseries
Map = geemap.Map()
Map.centerObject(bounds, zoom=12)
Map.ts_inspector(left_ts=sentinel_ts, right_ts=sentinel_ts, 
                 left_names=layer_names, right_names=layer_names,
                 left_vis=sentinel_vis, right_vis=sentinel_vis)
Map

Map(center=[-8.033989020419423, 112.93390157219604], controls=(WidgetControl(options=['position'], widget=Drop…

In [53]:
#Membuat Variabel
img1_srtdt = input("Tanggal Mulai Kondisi 1:")
img1_enddt = input("Tanggal Akhir Kondisi 1:")

img2_srtdt = input("Tanggal Mulai Kondisi 2:")
img2_enddt = input("Tanggal Akhir Kondisi 2:")

image1 = imagecol.filterDate(img1_srtdt, img1_enddt).median()
image2 = imagecol.filterDate(img2_srtdt, img2_enddt).median()



#Tile Images Layers
Layer1 = geemap.ee_tile_layer(image1.clip(bounds),
                                 {'bands':['B4', 'B3','B2'],
                                  'min':0, 'max':2500},
                                 name='ImageCollection1')
Layer2 = geemap.ee_tile_layer(image2.clip(bounds),
                                 {'bands':['B4', 'B3','B2'],
                                  'min':0, 'max':2500},
                                 name='ImageCollection2')

#Tile Indeks  Vegetasi Layer
NDVI1 = 

Tanggal Mulai Kondisi 1:2017-03-01
Tanggal Akhir Kondisi 1:2017-08-30
Tanggal Mulai Kondisi 2:2019-03-01
Tanggal Akhir Kondisi 2:2019-08-30


In [55]:
Map = geemap.Map()
Map.centerObject(bounds, zoom=12)
Map.split_map(Layer1, Layer2)

Map

Map(center=[-8.033989020419423, 112.93390157219604], controls=(WidgetControl(options=['position'], widget=HBox…

In [48]:
#Draw Features
feature = Map.draw_features
polygon = ee.Geometry.MultiPolygon(feature)

In [34]:
#Export Image
geemap.ee_export_image(NDVI,
                      filename='NDVI.tif', scale=10,
                      region=feature.geometry())

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'D:\\Cloud Computing\\NDVI.zip'


In [ ]:
imagecol.getDownloadUrl({'name': 'TNBTS', 
                               'bands':['B4', 'B3', 'B2'], 
                               'region': bounds.geometry(), 
                               'scale':10, 
                               'filePerBand': False})